In [1]:
!pip install pandas numpy scikit-learn tensorflow==2.12.0 nltk tldextract


  Using cached tensorflow-2.12.0-cp310-cp310-win_amd64.whl.metadata (2.5 kB)
  Using cached tensorflow_intel-2.12.0-cp310-cp310-win_amd64.whl.metadata (4.1 kB)
  Using cached jax-0.6.2-py3-none-any.whl.metadata (13 kB)
  Using cached tensorboard-2.12.3-py3-none-any.whl.metadata (1.8 kB)
  Using cached google_auth-2.47.0-py3-none-any.whl.metadata (6.4 kB)
  Using cached google_auth_oauthlib-1.0.0-py2.py3-none-any.whl.metadata (2.7 kB)
  Using cached jaxlib-0.6.2-cp310-cp310-win_amd64.whl.metadata (1.4 kB)
INFO: pip is looking at multiple versions of jax to determine which version is compatible with other requirements. This could take a while.
  Using cached jax-0.6.1-py3-none-any.whl.metadata (13 kB)
  Using cached jaxlib-0.6.1-cp310-cp310-win_amd64.whl.metadata (1.2 kB)
  Using cached jax-0.6.0-py3-none-any.whl.metadata (22 kB)
  Using cached jaxlib-0.6.0-cp310-cp310-win_amd64.whl.metadata (1.2 kB)
  Using cached jax-0.5.3-py3-none-any.whl.metadata (22 kB)
  Using cached jaxlib-0.5.3-c

In [1]:
import pandas as pd
import numpy as np
import re
import nltk
import tldextract

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import accuracy_score

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Embedding
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

nltk.download('stopwords')


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\ADMIN\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [2]:
# Files are inside data/ folder
url_df = pd.read_csv("data/phishing.csv")
email_df = pd.read_csv("data/mail_data.csv")

url_df.head(), email_df.head()


(   Index  UsingIP  LongURL  ShortURL  Symbol@  Redirecting//  PrefixSuffix-  \
 0      0        1        1         1        1              1             -1   
 1      1        1        0         1        1              1             -1   
 2      2        1        0         1        1              1             -1   
 3      3        1        0        -1        1              1             -1   
 4      4       -1        0        -1        1             -1             -1   
 
    SubDomains  HTTPS  DomainRegLen  ...  UsingPopupWindow  IframeRedirection  \
 0           0      1            -1  ...                 1                  1   
 1          -1     -1            -1  ...                 1                  1   
 2          -1     -1             1  ...                 1                  1   
 3           1      1            -1  ...                -1                  1   
 4           1      1            -1  ...                 1                  1   
 
    AgeofDomain  DNSRecording 

In [3]:
# URL features & target
X_url = url_df.drop(columns=['Index','class'])
y_url = url_df['class']

# Convert -1/1 to 0/1 for sklearn
y_url = y_url.map({1:0, -1:1})  # 0=safe, 1=phishing

X_url.head()


,UsingIP,LongURL,ShortURL,Symbol@,Redirecting//,PrefixSuffix-,SubDomains,HTTPS,DomainRegLen,Favicon,...,DisableRightClick,UsingPopupWindow,IframeRedirection,AgeofDomain,DNSRecording,WebsiteTraffic,PageRank,GoogleIndex,LinksPointingToPage,StatsReport
0,1,1,1,1,1,-1,0,1,-1,1,...,1,1,1,-1,-1,0,-1,1,1,1
1,1,0,1,1,1,-1,-1,-1,-1,1,...,1,1,1,1,-1,1,-1,1,0,-1
2,1,0,1,1,1,-1,-1,-1,1,1,...,1,1,1,-1,-1,1,-1,1,-1,1
3,1,0,-1,1,1,-1,1,1,-1,1,...,1,-1,1,-1,-1,0,-1,1,1,1
4,-1,0,-1,1,-1,-1,1,1,-1,1,...,1,1,1,1,1,1,-1,1,-1,-1


In [4]:
# Email labels
email_df['label'] = email_df['Category'].map({'ham':0,'spam':1})
X_email_text = email_df['Message']
y_email = email_df['label']

X_email_text.head()


0    Go until jurong point, crazy.. Available only ...
1                        Ok lar... Joking wif u oni...
2    Free entry in 2 a wkly comp to win FA Cup fina...
3    U dun say so early hor... U c already then say...
4    Nah I don't think he goes to usf, he lives aro...
Name: Message, dtype: object

In [5]:
X_train, X_test, y_train, y_test = train_test_split(
    X_url, y_url, test_size=0.2, random_state=42
)

url_ml = RandomForestClassifier(n_estimators=100)
url_ml.fit(X_train, y_train)

pred = url_ml.predict(X_test)
print("URL ML Accuracy:", accuracy_score(y_test, pred))


URL ML Accuracy: 0.9678878335594754


In [6]:
url_dl = Sequential()
url_dl.add(Dense(64, activation='relu', input_shape=(X_url.shape[1],)))
url_dl.add(Dense(32, activation='relu'))
url_dl.add(Dense(1, activation='sigmoid'))

url_dl.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics=['accuracy']
)

url_dl.fit(
    X_train, y_train,
    epochs=10,
    batch_size=32,
    validation_split=0.2
)


Epoch 1/10
222/222 [==============================] - 4s 7ms/step - loss: 0.3029 - accuracy: 0.8793 - val_loss: 0.1866 - val_accuracy: 0.9288
Epoch 2/10
222/222 [==============================] - 1s 2ms/step - loss: 0.1793 - accuracy: 0.9299 - val_loss: 0.1679 - val_accuracy: 0.9339
Epoch 3/10
222/222 [==============================] - 1s 3ms/step - loss: 0.1637 - accuracy: 0.9365 - val_loss: 0.1572 - val_accuracy: 0.9378
Epoch 4/10
222/222 [==============================] - 1s 3ms/step - loss: 0.1504 - accuracy: 0.9395 - val_loss: 0.1461 - val_accuracy: 0.9373
Epoch 5/10
222/222 [==============================] - 1s 4ms/step - loss: 0.1412 - accuracy: 0.9422 - val_loss: 0.1445 - val_accuracy: 0.9406
Epoch 6/10
222/222 [==============================] - 1s 3ms/step - loss: 0.1320 - accuracy: 0.9440 - val_loss: 0.1398 - val_accuracy: 0.9395
Epoch 7/10
222/222 [==============================] - 1s 3ms/step - loss: 0.1219 - accuracy: 0.9505 - val_loss: 0.1235 - val_accuracy: 0.9480
Epoch 

In [7]:
tfidf = TfidfVectorizer(stop_words='english', max_features=3000)
X_email = tfidf.fit_transform(X_email_text)

Xe_train, Xe_test, ye_train, ye_test = train_test_split(
    X_email, y_email, test_size=0.2, random_state=42
)

email_ml = LogisticRegression(max_iter=1000)
email_ml.fit(Xe_train, ye_train)

email_pred = email_ml.predict(Xe_test)
print("Email ML Accuracy:", accuracy_score(ye_test, email_pred))


Email ML Accuracy: 0.967713004484305


In [8]:
tokenizer = Tokenizer(num_words=5000)
tokenizer.fit_on_texts(X_email_text)

sequences = tokenizer.texts_to_sequences(X_email_text)
padded = pad_sequences(sequences, maxlen=200)

Xe_tr, Xe_te, ye_tr, ye_te = train_test_split(
    padded, y_email, test_size=0.2, random_state=42
)


In [9]:
email_dl = Sequential()
email_dl.add(Embedding(5000, 64, input_length=200))
email_dl.add(LSTM(64))
email_dl.add(Dense(1, activation='sigmoid'))

email_dl.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics=['accuracy']
)

email_dl.fit(Xe_tr, ye_tr, epochs=5, batch_size=32, validation_split=0.2)


Epoch 1/5
112/112 [==============================] - 34s 253ms/step - loss: 0.2351 - accuracy: 0.9229 - val_loss: 0.0954 - val_accuracy: 0.9697
Epoch 2/5
112/112 [==============================] - 26s 235ms/step - loss: 0.0406 - accuracy: 0.9879 - val_loss: 0.0614 - val_accuracy: 0.9843
Epoch 3/5
112/112 [==============================] - 24s 219ms/step - loss: 0.0156 - accuracy: 0.9969 - val_loss: 0.0625 - val_accuracy: 0.9854
Epoch 4/5
112/112 [==============================] - 26s 229ms/step - loss: 0.0068 - accuracy: 0.9980 - val_loss: 0.0849 - val_accuracy: 0.9787
Epoch 5/5
112/112 [==============================] - 24s 213ms/step - loss: 0.0136 - accuracy: 0.9961 - val_loss: 0.0740 - val_accuracy: 0.9821


In [10]:
def predict_url(input_features):
    features = np.array([list(input_features.values())])

    ml_prob = url_ml.predict_proba(features)[0][1]
    dl_prob = url_dl.predict(features)[0][0]

    final_prob = (ml_prob + dl_prob) / 2

    suspicious_words = ['login','verify','account','secure','update','bank','paypal']
    url_feature_text = " ".join(input_features.keys()).lower()
    keyword_flag = any(word in url_feature_text for word in suspicious_words)

    if ml_prob > 0.3 or dl_prob > 0.3 or keyword_flag:
        verdict = "PHISHING URL 🚨"
    else:
        verdict = "SAFE URL ✅"

    return {
        "Verdict": verdict,
        "ML_Phishing_%": round(ml_prob * 100, 2),
        "DL_Phishing_%": round(dl_prob * 100, 2),
        "Final_Confidence_%": round(final_prob * 100, 2)
    }


In [23]:
feature_names = np.array(tfidf.get_feature_names_out())

def explain_email(text, top_n=5):
    vec = tfidf.transform([text])
    scores = vec.toarray()[0]
    top_indices = scores.argsort()[-top_n:][::-1]
    keywords = feature_names[top_indices]
    return keywords.tolist()

def predict_email_zero_trust_keywords(text):
    keywords = explain_email(text, top_n=5)
    
    ml_vec = tfidf.transform([text])
    ml_prob = email_ml.predict_proba(ml_vec)[0][1]
    
    seq = tokenizer.texts_to_sequences([text])
    padded_seq = pad_sequences(seq, maxlen=200)
    dl_prob = email_dl.predict(padded_seq)[0][0]
    
    final_prob = (ml_prob + dl_prob) / 2

    critical_words = ['urgent','verify','bank','account','click','password','suspend','login','immediately']
    verdict = "PHISHING EMAIL 🚨" if ml_prob>0.5 or dl_prob>0.5 or any(word in text.lower() for word in critical_words) else "SAFE EMAIL ✅"

    return {
        "Verdict": verdict,
        "ML_Phishing_%": round(ml_prob*100,2),
        "DL_Phishing_%": round(dl_prob*100,2),
        "Final_Confidence_%": round(final_prob*100,2),
        "Suspicious_Keywords": keywords
    }


In [11]:
from urllib.parse import urlparse

def extract_url_features(url):
    features = {}
    features['UsingIP'] = 1 if re.match(r'http[s]?://\d+\.\d+\.\d+\.\d+', url) else 0
    features['LongURL'] = 1 if len(url) > 75 else 0
    features['ShortURL'] = 1 if re.search(r'tinyurl|bit\.ly|goo\.gl', url) else 0
    features['Symbol@'] = 1 if '@' in url else 0
    features['Redirecting//'] = 1 if urlparse(url).path.count('//') > 0 else 0
    features['PrefixSuffix-'] = 1 if '-' in urlparse(url).netloc else 0
    features['SubDomains'] = 1 if urlparse(url).netloc.count('.') > 2 else 0
    features['HTTPS'] = 1 if urlparse(url).scheme == 'https' else 0
    features['DomainRegLen'] = 1 if len(urlparse(url).netloc) < 10 else 0

    other_features = ['Favicon','Port','HTTPSdomain','RequestURL','URLofAnchor','LinksInScriptTags','SFH',
                      'SubmittingToEmail','AbnormalURL','Redirect','OnMouseOver','RightClick','PopUpWidnow',
                      'Iframe','AgeofDomain','DNSRecord','WebsiteTraffic','PageRank','GoogleIndex',
                      'LinksPointingToPage','StatsReport']
    
    for feat in other_features:
        features[feat] = 0

    return features


In [ ]:
from urllib.parse import urlparse

def extract_url_features(url):
    features = {}
    features['UsingIP'] = 1 if re.match(r'http[s]?://\d+\.\d+\.\d+\.\d+', url) else 0
    features['LongURL'] = 1 if len(url) > 75 else 0
    features['ShortURL'] = 1 if re.search(r'tinyurl|bit\.ly|goo\.gl', url) else 0
    features['Symbol@'] = 1 if '@' in url else 0
    features['Redirecting//'] = 1 if urlparse(url).path.count('//') > 0 else 0
    features['PrefixSuffix-'] = 1 if '-' in urlparse(url).netloc else 0
    features['SubDomains'] = 1 if urlparse(url).netloc.count('.') > 2 else 0
    features['HTTPS'] = 1 if urlparse(url).scheme == 'https' else 0
    features['DomainRegLen'] = 1 if len(urlparse(url).netloc) < 10 else 0
    
    other_features = ['Favicon','Port','HTTPSdomain','RequestURL','URLofAnchor',
                      'LinksInScriptTags','SFH','SubmittingToEmail','AbnormalURL','Redirect',
                      'OnMouseOver','RightClick','PopUpWidnow','Iframe','AgeofDomain','DNSRecord',
                      'WebsiteTraffic','PageRank','GoogleIndex','LinksPointingToPage','StatsReport']
    
    for feat in other_features:
        features[feat] = 0  # Placeholder for demo
    
    return features


In [12]:
input_type = input("Enter 'url' to check URL or 'email' to check Email: ").lower()

if input_type == 'url':
    url_input = input("Enter full URL: ")
    features_dict = extract_url_features(url_input)
    result = predict_url(features_dict)
    print("\n🔐 ZERO‑TRUST URL DECISION")
    for k,v in result.items():
        print(f"{k}: {v}")

elif input_type == 'email':
    email_input = input("Enter Email content: ")
    result = predict_email_zero_trust_keywords(email_input)
    print("\n🔐 ZERO‑TRUST EMAIL DECISION")
    for k,v in result.items():
        print(f"{k}: {v}")

else:
    print("Invalid input. Enter 'url' or 'email'.")


Enter 'url' to check URL or 'email' to check Email:  url
Enter full URL:  https:


C:\Users\ADMIN\OneDrive\Desktop\phishing_project\venv310\lib\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(


1/1 [==============================] - 0s 234ms/step

🔐 ZERO‑TRUST URL DECISION
Verdict: PHISHING URL 🚨
ML_Phishing_%: 38.0
DL_Phishing_%: 1.07
Final_Confidence_%: 19.53


In [13]:
import joblib
import os

os.makedirs("models", exist_ok=True)

joblib.dump(url_ml, "models/url_ml.pkl")
joblib.dump(email_ml, "models/email_ml.pkl")
joblib.dump(tfidf, "models/tfidf.pkl")
joblib.dump(tokenizer, "models/tokenizer.pkl")

url_dl.save("models/url_dl.h5")
email_dl.save("models/email_dl.h5")


In [14]:
from tensorflow.keras.preprocessing.text import Tokenizer


In [15]:
import joblib
joblib.dump(tokenizer, "tokenizer_tf.pkl")


['tokenizer_tf.pkl']

In [17]:
from tensorflow.keras.preprocessing.text import Tokenizer
import joblib

# Assuming `X_email_text` is your training data
tokenizer = Tokenizer(num_words=5000)
tokenizer.fit_on_texts(X_email_text)

# Save tokenizer to a file
joblib.dump(tokenizer, "models/tokenizer_tf.pkl")


['models/tokenizer_tf.pkl']

In [19]:
import joblib

# Assuming 'tokenizer' is your trained Tokenizer object
joblib.dump(tokenizer, "models/tokenizer_tf.pkl")
print("Tokenizer saved successfully!")


Tokenizer saved successfully!


In [20]:
tokenizer = joblib.load("models/tokenizer_tf.pkl")
